In [1]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

import dtale

# notice we are adding two new imports for visualizations; This was not here last class
from itertools import product
from pandas_profiling import ProfileReport


# for better resolution plots
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()


from datetime import datetime
import calendar

In [2]:
data = pd.read_csv ("/Users/vsa/Documents/DM/data/donors.csv", low_memory=False)

In [20]:
# Choosing columns of interest 
col_df = data[['STATE',
               'ZIP',
               'MAILCODE',
               'PVASTATE',
               'DOB',
               'DOMAIN',
               'HOMEOWNR',
               'NUMCHLD',
               'INCOME',
               'GENDER',
               'WEALTH1', 
               'LOCALGOV',
               'STATEGOV',
               'FEDGOV',
               'SOLIH',
               'WEALTH2', #HC (highly correlated) with WEALTH1 (0.86) - could be used to fill nans and drop 1
               'VETERANS',
               'AGE901',
 #              'HHP1', #HC (highly correlated) with RHP3 (0.97)
               'HV1',
               'HV3',
               'RHP1',
               'RHP3',
               'RHP4',
               'MSA',
               'DMA',
               'IC1',
               'IC2',
               'IC15',
               'IC16',
               'IC17',
               'IC18',
               'IC19',
               'IC20',
               'IC21',
               'IC22',
               'IC23',
#               'OEDC1', # HC with LOCALGOV 0.82
#               'OEDC2', # HC with STATEGOV 0.85
#               'OEDC3', # HC with FEDGOV 0.86
               'OEDC4',
               'OEDC5',
               'OEDC6',
               'OEDC7',
               'EC1',
               'AC1',
               'AC2',
               'RFA_2R',
               'RFA_2F',
               'RFA_2A',
               
               'RFA_3', #need to be split to 3 columns as RFA 2, has additional info about status from 2016
               
               'MDMAUD_R',
               'MDMAUD_F',
               'MDMAUD_A',
              # 'CONTROLN',
               'CARDPROM', 
#               'NUMPROM', #HC with CARDPROM - 0.95
               'RAMNTALL',
 #              'NGIFTALL',# HC with CARDGIFT, no need to have an info about all gifts if we explore card response
               'CARDGIFT',
               'LASTGIFT',
               'AVGGIFT',
               'CARDPM12',
               'NUMPRM12',
               'PEPSTRFL']]

In [21]:
df = col_df.iloc[:15000, :].copy()

In [22]:
# Dimension of the dataset
df.shape

(15000, 58)

In [11]:
# Check duplicates in data
any(data.duplicated())

False

In [13]:
#Number of missing values in DOB
len(data[data.DOB.isnull()])

23883

In [ ]:
# df[~df.DOB.isnull()]

In [19]:
# df.DOB.fillna('1800-01-01', inplace=True)
# df.DOB  = df.DOB.map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# df['YOB']  = df.DOB.map(lambda x: x.year)

In [23]:
dtale.show(df)

## Coherence Checks
Date of donor's first gift to PVA
cannot be earlier than his/her Date of Birthday (DOB)

In [39]:
len(data[data.ODATEDW <= data.DOB])

318

In [46]:
# df[df.PEPSTRFL!='X'].AVGGIFT.mean()

15.988760685064387

In [47]:
# df[df.PEPSTRFL=='X'].AVGGIFT.mean()

10.33432021457179